## Mapping 

## Generate Numbers with a GAN

# GAN Image

Here Z is a noise vector - random numbers<br/>
Here Y is the probability - The value of Y will be compared to 1 and Re-Train



## 1 - Value of Real Image, 0 - Value of Fake Image

In Discriminator both Fake and Real image will be provided and Y will be compared for both Fake vs Fake Label(0) and Real vs Real Label(1) - re-Train - BackPropagation - automatically done by PyTorch

## Cross Entropy

[Cross Entropy](https://www.youtube.com/watch?v=FODwUM-1PyI)

[Expected Value](https://www.youtube.com/watch?v=KLs_7b7SKi4)

For Information Theory we use Log

How to use Cross Entropy Function
Example - a person travelling in Titanic - Survived - 1 or Died - 0


Wanted Probability - p(x) = [1, 0] - That actually happened<br/>
Actual Probability - q(x) = [0.4, 0.6] - Generated by the model we will use Cross Entropy to calculate how far it is from the Wanted Probability<br/>


<!-- $$ H(p, q) = -\sum_{i=1}^{n} p(x) * log q(x)$$ -->

$$ H(p, q) = -\sum p(x) * log q(x)$$ 


H = -[1*ln(0.4)+0*ln(0.6)] = 0.91, more the value - more far from the wanted probability















In [3]:
# Import the libraries
import torch, pdb # Here pdb is the library to do debugging in Python 
# Dataloader will be able to iterate through the training data as we train the network
from torch.utils.data import DataLoader # Dataloader is a library of pytorch
from torch import nn # nn is part of Pytorch library that helps us build very quickly deep learning architectures and models
#transforms will allow us to transform out training data in different ways
from torchvision import  transforms# torchvision is for the computer vision
from torchvision.datasets import MNIST
#We will build a grid of images to evaluate during the training - how the fake images created by the generator
from torchvision.utils import make_grid

#library to create a beautiful progressbar as you are training
from tqdm.auto import tqdm 


import matplotlib.pyplot as plt 



/home/deb/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Create a function - to display as we are training 
# - agreed with the number of generated images versus the real images

# Here ch = 1 is channel (grayscale)
# size = (28, 28) size of the MNIST dataset
# num = number of images we want to show - grid of 4x4 or 5x5
# Visualization function
def show(tensor, ch=1, size=(28, 28), num=16):
    #tensor: 128x784
    # when using backpropagation in pytorch, gradients are being calculated when they are linked to different
    # variables - we want to just visualize whats contained in that variable - don't want any processing to take place 
    # with a gradient. detach that variable from the computation of gradients
    # size = 28x28 - reshaping that we are doing
    # tensor = 128 x 784 - we will process 128 images at a time, 784 = 28x28
    # The tensor that we will be managing during the training - is going to have this dimensions -  128 x 784
    # ch will be 1
    # whatever remains will be the first dimension
    data = tensor.detach().cpu().view(-1, ch, *size) # 128 x 1 x 28 x 28
    # We want cpu only for the visualization
    # result of this operation is going to transform this 128x784 dimensional tensor into a -
    # 128 x 1 x 28 x 28, size = 28, ch = 1, whatever remains -1

    # We are going to take the first 16 elements of the above multidimensional tensor that has 128
    # From that 128 we are goint to take 16
    # The second parameter - number of rows we want to set
    # nrows - 16 pictures - sq. grid -4 rows x 4 columns 
    # permute is a function that allows us to change the order of the channels - order of the dimension
    
    grid = make_grid(data[:num], nrows=4).permute(1, 2, 0) # pytorch - channel(C), width(W), height(H) nut matplotlib WHC (1, 2, 0) are indeces > 28x28x1
    
    # Why? The first computation will give [16 x 1 x 28 x 28]
    # we will get a series of images that have this internal structure - [16 x 1 x 28 x 28]
    # But if we want to display these with matplotlib the author of the channel has to be different pytorch uses
    # channel with width and height - but we want matplotlib - to have width, height and channel - so change the order of the channels

    

    plt.imshow(grid)
    plt.show()

       

## What we have dane since now:
1. We imported the libnraries
2. We created the visualization function - to see later the content of the image created by generated during the training process

In [ ]:
# Setup of main parameters and hyperparameters 
epochs = 500 # no. of cycles we are going to perform during the training 
cur_step = 0 # In each of the steps of the training we are going to process 1 batch - 1 set of images
info_step = 300# This is going to store every how many steps we want to show in the screen information about the current LOSS values
# We need to show the generator loss and discriminator loss

mean_gen_loss = 0
mean_disc_loss = 0

#Hyper parameters
z_dim = 50 # dimensionality of the noise vector, input of the generator
# value can be anything what will be the dimentionality  of the latent space that we are going to learn
lr = 0.00001 # learning rate, we will use adam optimizer
loss_func = nn.BCEWithLogitsLoss() # Binary Cross entropy
# 1. set the values 0 to 1; 2. After then Binary Cross Entropy
# It is going to take the output of the neural network that are the logists and than 
# that function is going to apply all at once a sigmoid function
# BCELoss vs BCEWithLogitsLoss - convert the numerical values to the range that goes from 0 to 1


bs = 128 #batch size = during each step of the training how many images are going to process at once in the GPU.
# Great adv. of GPU is that we can send a number of images in parallel to be processed by the GPU 


device = 'cuda' # Device where we are going to process the data

dataloader = DataLoader(MNIST('.', download=True, transform=transforms.ToTensor())) # Hold our training data - give us back a bunch of training data containing 
# input elements for a network and a set of labels for those elements
# . stor it in root
# Transform the MNIST to a multi-dimensional tensor




